In [1]:
import numpy as np
import pandas as pd

In [2]:
from scipy.stats import norm

In [3]:
def black_scholes_call(S,K,sigma,r,T):
  d1=(np.log(S/K)+(r+(sigma**2)/2)*T)/(sigma*np.sqrt(T))
  d2=d1-sigma*np.sqrt(T)
  return S*norm.cdf(d1)-K*np.exp(-r*T)*norm.cdf(d2)

In [4]:
def delta(S,K,sigma,r,T):
  d1=(np.log(S/K)+(r+(sigma**2)/2)*T)/(sigma*np.sqrt(T))
  return norm.cdf(d1)

## heding delta via binomial

In [6]:
def delta_hedge_call(S,K,sigma,r,T,N):
  dt=T/N
  u=np.exp(sigma*np.sqrt(dt))
  d=1/u
  p=(np.exp(r*dt)-d)/(u-d)

  #generating stock price tree
  stock_tree=np.zeros((N+1,N+1))
  for i in range(N+1):
    for j in range(i+1):
      stock_tree[j,i]=S*(u**(i-j))*(d**j)

  #generating option value tree
  option_tree=np.zeros((N+1,N+1))
  for j in range(N+1):
    option_tree[j,N]=np.maximum(stock_tree[j,N]-K,0)
  for i in range(N-1,-1,-1):
    for j in range(i+1):
      option_tree[j,i]=np.exp(-r*dt)*(p*option_tree[j,i+1]+(1-p)*option_tree[j+1,i+1])

  #simulating delta hedge
  hedge=[]
  stock_position=delta(S,K,sigma,r,T)*S
  option_price=option_tree[0,0]
  hedge.append(-option_price)
  for i in range(N+1):
    stock_price=stock_tree[0,i]

In [11]:
import numpy as np

# Define the input values
S0 = 100      # Initial stock price
u = 1.3499    # Up factor
d = 0.7408    # Down factor
p = 0.5097    # Probability of up move

# Define the number of time steps and initialize the stock price tree
n = 3
stock_tree = np.zeros((n+1, n+1))
stock_tree[0, 0] = S0

# Fill in the stock price tree
for i in range(1, n+1):
    for j in range(i+1):
        stock_tree[j, i] = S0 * u**(i-j) * d**j

# Print the stock price tree
print("Stock price tree:")
print(stock_tree)

# Initialize the option price tree
option_tree = np.zeros((n+1, n+1))

# Fill in the option price tree at the final nodes
for j in range(n+1):
    option_tree[j, n] = max(stock_tree[j, n] - 100, 0)

# Work backwards through the tree to calculate the option prices at each node
for i in range(n-1, -1, -1):
    for j in range(i+1):
        option_tree[j, i] = np.exp(-0.06 * 1/3) * (p * option_tree[j, i+1] + (1-p) * option_tree[j+1, i+1])

# Print the option price tree
print("Option price tree:")
print(option_tree)

Stock price tree:
[[100.         134.99       182.223001   245.98282905]
 [  0.          74.08       100.000592   134.99079914]
 [  0.           0.          54.878464    74.08043855]
 [  0.           0.           0.          40.65396613]]
Option price tree:
[[ 30.79729065  53.24146579  89.75035926 145.98282905]
 [  0.           8.73396303  17.48165742  34.99079914]
 [  0.           0.           0.           0.        ]
 [  0.           0.           0.           0.        ]]
Option Price
30.79729064682631


In [16]:
import numpy as np

# Define the input values
S0 = 100      # Initial stock price
u = 1.3499       # Up factor
d = 0.7408       # Down factor
r = 0.06      # Risk-free interest rate
T = 1         # Time to maturity (in years)
K = 105       # Strike price

# Define the number of time steps and initialize the stock price tree
n = 3
dt = T / n
stock_tree = np.zeros((n+1, n+1))
stock_tree[0, 0] = S0

# Fill in the stock price tree
for i in range(1, n+1):
    for j in range(i+1):
        stock_tree[j, i] = S0 * u**(i-j) * d**j

# Initialize the option price tree
option_tree = np.zeros((n+1, n+1))

# Fill in the option price tree at the final nodes
for j in range(n+1):
    option_tree[j, n] = max(stock_tree[j, n] - K, 0)

# Work backwards through the tree to calculate the option prices at each node
for i in range(n-1, -1, -1):
    for j in range(i+1):
        option_tree[j, i] = np.exp(-r * dt) * (u * option_tree[j, i+1] + d * option_tree[j+1, i+1]) / 2

# Print the option price at time 0
print("Option price:", option_tree[0, 0])

Option price: 55.12242105218408
